### Effet d'information

Il est essentiel de bien comprendre la notion d'information dans le contexte de l’estimation des ressources minières. En effet, plus on dispose de données fiables, plus on obtient une image précise du gisement. Mais il ne suffit pas d’avoir beaucoup de données : encore faut-il qu’elles soient de qualité, c’est-à-dire **précises** et **exactes**.

- **La précision** décrit la répétabilité des mesures. Par exemple, si l’analyse d’une même carotte donne des résultats très proches lorsqu’elle est répétée, la précision est bonne. Cela signifie que les erreurs aléatoires sont faibles.

- **L’exactitude** mesure à quel point les données sont proches de la valeur réelle. Si une méthode d’analyse ou d’estimation produit toujours une surestimation ou une sous-estimation, elle souffre d’un **biais systématique**.

Lorsque les données sont à la fois précises et exactes, on dit qu’elles sont **justes**.

Dans la pratique, les teneurs sont estimées à partir de données comme les forages et les analyses en laboratoire, mais la valeur réelle ne peut être confirmée qu’après l’extraction du minerai. De plus, on ne réalise pas 50 analyses sur la même carotte : on compare plutôt les teneurs estimées (une valeur par localisation) avec les teneurs réellement mesurées après extraction, afin d’évaluer la présence d’erreurs ou de biais.

Les écarts entre les teneurs estimées et les teneurs réelles peuvent être dus à la **variabilité des mesures** (manque de précision) ou à un **biais systématique dans la méthode d’estimation** (manque d’exactitude).

Avec ce graphique interactif, vous pouvez explorer comment la précision et l’exactitude influencent les résultats d’estimation. Amusez-vous à comparer l'effet du bruit aléatoire ou du biais systématique sur les cartes de teneur.

In [1]:

import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft2, ifft2, fftshift
import ipywidgets as widgets
from ipywidgets import interact

def spherical_covariance_fft(size, range_, sill=1.0):
    extended_size = 2 * size
    x = np.arange(-extended_size//2, extended_size//2)
    X, Y = np.meshgrid(x, x)
    h = np.sqrt(X**2 + Y**2)
    h = np.minimum(h, range_ * size)
    cov = sill * (1 - 1.5 * (h / (range_ * size)) + 0.5 * (h / (range_ * size))**3)
    cov[h > (range_ * size)] = 0
    return fftshift(cov)

def fftma_simulation(size, range_, sill=1.0, seed=0):
    np.random.seed(seed)
    extended_size = 2 * size
    cov_model = spherical_covariance_fft(size, range_, sill)
    cov_fft = fft2(cov_model)
    white_noise = np.random.normal(size=(extended_size, extended_size))
    white_fft = fft2(white_noise)
    z_fft = np.sqrt(np.abs(cov_fft)) * white_fft
    z_ext = np.real(ifft2(z_fft))
    start = extended_size // 4
    end = start + size
    return z_ext[start:end, start:end]

def gaussian_to_lognormal(field):
    return np.exp(field)

def plot_real_vs_estimated_bias(bias_percent=10, noise_std=0.5, cutoff=2):
    size = 200
    seed = 42  # On fixe la graine pour la reproductibilité
    np.random.seed(seed)
    
    field = fftma_simulation(size=size, range_=0.2, sill=1.0, seed=seed)
    real = gaussian_to_lognormal(field)
    
    # Biais systématique multiplicatif : on ajoute X% de la valeur réelle à la valeur réelle
    biased_real = real * (1 + bias_percent / 100)
    
    # Bruit aléatoire normal centré sur la valeur réelle, écart-type noise_std
    noise = np.random.normal(loc=0, scale=noise_std, size=real.shape)
    
    # Champ estimé = valeur biaisée + bruit (le bruit peut augmenter ou diminuer)
    estimated = biased_real + noise
    
    lower_clip, upper_clip = 0, 10
    real_clipped = np.clip(real, lower_clip, upper_clip)
    estimated_clipped = np.clip(estimated, lower_clip, upper_clip)
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    im0 = axes[0].imshow(real_clipped, cmap='viridis', vmin=lower_clip, vmax=upper_clip)
    axes[0].set_title("Champ réel")
    axes[0].set_xlabel("X (m)")
    axes[0].set_ylabel("Y (m)")
    fig.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)

    im1 = axes[1].imshow(estimated_clipped, cmap='viridis', vmin=lower_clip, vmax=upper_clip)
    axes[1].set_title("Champ estimé")
    axes[1].set_xlabel("X (m)")
    axes[1].set_ylabel("Y (m)")
    fig.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)

    real_flat = real_clipped.flatten()
    estimated_flat = estimated_clipped.flatten()

    # Masques selon la description :
    # Minerai ignoré : réel ≥ cutoff ET estimé < cutoff → rouge
    mask_red = (real_flat >= cutoff) & (estimated_flat < cutoff)
    # Stérile traité : réel < cutoff ET estimé ≥ cutoff → bleu
    mask_blue = (real_flat < cutoff) & (estimated_flat >= cutoff)
    # Les autres points gris
    mask_other = ~(mask_red | mask_blue)

    # Nuage de points
    axes[2].scatter(estimated_flat[mask_other], real_flat[mask_other], alpha=0.3, s=10, color="gray", edgecolor="none", label="Correctement classé")
    axes[2].scatter(estimated_flat[mask_blue], real_flat[mask_blue], alpha=0.7, s=15, color="blue", edgecolor="k", linewidth=0.2, label="Stérile traité")
    axes[2].scatter(estimated_flat[mask_red], real_flat[mask_red], alpha=0.7, s=15, color="red", edgecolor="k", linewidth=0.2, label="Minerai ignoré")

    axes[2].plot([lower_clip, upper_clip], [lower_clip, upper_clip], 'k-', linewidth=2, label="Ligne 1:1")

    # Régression linéaire inversée : y=réel en fonction de x=estimé
    A = np.vstack([estimated_flat, np.ones_like(estimated_flat)]).T
    a, b = np.linalg.lstsq(A, real_flat, rcond=None)[0]
    x_fit = np.array([lower_clip, upper_clip])
    y_fit = a * x_fit + b
    axes[2].plot(x_fit, y_fit, 'r--', label=f"Régression linéaire\nréel={a:.2f}*estimé+{b:.2f}")

    # Lignes de coupure verticale et horizontale
    axes[2].axhline(cutoff, color='gray', linestyle='--')
    axes[2].axvline(cutoff, color='gray', linestyle='--')

    # Flèche horizontale démarrant sur la ligne verticale cutoff, au niveau cutoff en Y
    x_arrow_start = cutoff
    y_arrow = 8  # sur la ligne de coupure horizontale
    arrow_length = 2  # longueur de la flèche

    axes[2].annotate('',
                     xy=(x_arrow_start + arrow_length, y_arrow),
                     xytext=(x_arrow_start, y_arrow),
                     arrowprops=dict(facecolor='black', shrink=0.05, width=2, headwidth=8))

    # Texte juste au-dessus de la flèche, à droite du cutoff
    text_x = x_arrow_start + arrow_length * 0.5
    text_y = y_arrow + 0.3

    axes[2].text(text_x, text_y, 'Traité', fontsize=12, ha='left', va='bottom')

    axes[2].set_xlabel("Teneur estimée (ppm)")
    axes[2].set_ylabel("Teneur réelle (ppm)")
    axes[2].set_title("Teneur réelle vs estimée")
    axes[2].set_xlim(lower_clip, upper_clip)
    axes[2].set_ylim(lower_clip, upper_clip)
    axes[2].set_aspect('equal', adjustable='box')

    # Pourcentage de points dans chaque quadrant (selon seuil cutoff)
    Q1 = np.sum((real_flat >= cutoff) & (estimated_flat >= cutoff))
    Q2 = np.sum(mask_red)   # minerai ignoré
    Q3 = np.sum((real_flat < cutoff) & (estimated_flat < cutoff))
    Q4 = np.sum(mask_blue)  # stérile traité
    total = len(real_flat)

    # Affichage des % dans les 4 coins
    axes[2].text(0.2, cutoff - 0.5, f'{Q1/total*100:.1f}%', fontsize=12)
    axes[2].text(0.2, 9.5, f'{Q2/total*100:.1f}%', fontsize=12, color='red')
    axes[2].text(cutoff + 0.2, 9.5, f'{Q3/total*100:.1f}%', fontsize=12)
    axes[2].text(8.5, 0.5, f'{Q4/total*100:.1f}%', fontsize=12, color='blue')

    axes[2].legend(loc='upper right')
    axes[2].grid(True, linestyle=':', alpha=0.5)
    plt.tight_layout()
    plt.show()

bias_slider = widgets.FloatSlider(
    value=0,
    min=-50,
    max=50,
    step=5,
    description='Biais systématique (%)',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '180px'}
)

noise_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=0.5,
    step=0.01,
    description='Écart-type bruit',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '180px'}
)

cutoff_slider = widgets.FloatSlider(
    value=2,
    min=2,
    max=8,
    step=0.1,
    description='Teneur de coupure (ppm)',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '180px'}
)

interact(plot_real_vs_estimated_bias,
         bias_percent=bias_slider,
         noise_std=noise_slider,
         cutoff=cutoff_slider);

interactive(children=(FloatSlider(value=0.0, description='Biais systématique (%)', layout=Layout(width='400px'…